# DeepEval Evaluation for RAG Responses

This notebook uses DeepEval for evaluation. The goal is to use the astrophysics-related questions, true answers, and RAG generated answers to evaluate the quality of responses using DeepEval.

## 1. Import libraries and dependencies

In [9]:
# !pip install deepeval
# !pip install ragas

In [10]:
import os
import pandas as pd
from datetime import datetime

from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.metrics import FaithfulnessMetric
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.metrics import ContextualRecallMetric
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.metrics import BiasMetric
from deepeval.metrics import ToxicityMetric
from deepeval.metrics import HallucinationMetric

from deepeval.metrics.ragas import RagasMetric
from deepeval.metrics.ragas import RAGASAnswerRelevancyMetric
from deepeval.metrics.ragas import RAGASFaithfulnessMetric
from deepeval.metrics.ragas import RAGASContextualRecallMetric
from deepeval.metrics.ragas import RAGASContextualPrecisionMetric

## 2. Load the RAG results

In [11]:
# load the dataset that has the question, true answer and RAG generated answer
trulens_cosinesimilarity_results_df = pd.read_csv("data/results/cosine_similarity_results_2025-03-15_22-29-45.csv")
trulens_cosinesimilarity_results_df.head()

question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                         true_answer  \
0  Quick comment on the code: \n \n \n \n  petarz...   
1  After several iteration with  @ktl  and  @rowe...   
2  I take this to mean that a DIASource which is ...   
3  Hi James \nmaybe \n dafButler.Butler.get_known...   
4  This has now been fixed and 3.3.4 is the curre...   

                                             context  \
0  Draft\nLVV-P106: Data Management Acceptance Te...   
1  In most cases, the SWIG files from the current...   
2  DPDD | LSE-163 | Latest Revision 2023-07-10\n1...   
3  3   Overview\nThe Butler is implemented as thr...   
4  LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use ...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

   trulens_Answer_Relevance  trulens_Groundedness  trulens_Context_Relevance  \
0                  0.000000              0.000000                   0.500000   
1                  0.666667              0.000000                   0.500000   
2                  1.000000              0.555556                   0.666667   
3                  1.000000              0.500000                   0.500000   
4                       NaN                   NaN                        NaN   

   cosine_Answer_Relevance  cosine_Groundedness  cosine_Context_Relevance  \
0                 0.159146             0.314478                  0.602906   
1                 0.545343             0.417874                  0.446407   
2                 0.810223             0.661697                  0.751487   
3                 0.446853             0.696487                  0.606108   
4                 0.297717             0.279427                  0.356063   

   similarity_to_ground_truth  
0                    0.159146  
1                    0.545343  
2                    0.810223  
3                    0.446853  
4                    0.297717

## 3. Get the DeepEval Metrics

Firstly, define the metrics

In [12]:
threshold07 = 0.7
threshold05 = 0.5
model = "gpt-3.5-turbo"
include_reason=True

In [13]:
# how relevant the RAG's response is compared to the provided question
answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=threshold07,
    model=model,
    include_reason=include_reason
)

# whether the RAG's response factually aligns with the contents of the context retrieved
faithfulness_metric = FaithfulnessMetric(
    threshold=threshold07,
    model=model,
    include_reason=include_reason
)

#  checks if the retrieved context is relevant to the question. 
#  it ranks relevant information higher and filters out irrelevant details
contextual_precision_metric = ContextualPrecisionMetric(
    threshold=threshold07,
    model=model,
    include_reason=include_reason
)

# extent to which the retrieval context aligns with the true answer
contextual_recall_metric = ContextualRecallMetric(
    threshold=threshold07,
    model=model,
    include_reason=include_reason
)

# evaluates the overall relevance of the information presented in retrieval context for a given question
contextual_relevancy_metric = ContextualRelevancyMetric(
    threshold=threshold07,
    model=model,
    include_reason=include_reason
)

# determine whether our RAG contains gender, racial, or political bias.
bias_metric = BiasMetric(
    threshold=threshold05,
    model=model,
    include_reason=include_reason
)

# evaluate toxicness in our RAG output. This is particularly useful for a fine-tuning use case.
toxicity_metric = ToxicityMetric(
    threshold=threshold05,
    model=model,
    include_reason=include_reason
)

# whether our RAG generates factually correct information by comparing the RAG's response to the retrieved context
hallucination_metric = HallucinationMetric(
    threshold=threshold05,
    model=model,
    include_reason=include_reason
)

# The RAGAS metric is the average of four distinct metrics:
#   RAGASAnswerRelevancyMetric
#   RAGASFaithfulnessMetric
#   RAGASContextualPrecisionMetric
#   RAGASContextualRecallMetric
# This metric provides a score to holistically evaluate of our RAG pipeline's generator and retriever
RAGAS_metric = RagasMetric(
    threshold=threshold05,
    model=model
)

# RAGAS  - Answer Relevancy Metric - how well the generated answer is semantically relevant to the question
RAGAS_answer_relevancy_metric = RAGASAnswerRelevancyMetric(
    threshold=threshold07,
    model=model
)

# RAGAS  - Faithfulness Metric - if the generated answer is truthful and grounded in the retrieved context
RAGAS_faithfulness_metric = RAGASFaithfulnessMetric(
    threshold=threshold07,
    model=model
)

# RAGAS  - Contextual Precision Metric -  whether the retrieved context contains only relevant information for answering the question
RAGAS_contextual_precision_metric = RAGASContextualPrecisionMetric(
    threshold=threshold07,
    model=model
)

# RAGAS  - Contextual Recall Metric - whether the retrieved context provides enough details to answer the question completely
RAGAS_contextual_recall_metric = RAGASContextualRecallMetric(
    threshold=threshold07,
    model=model
)

metrics=[answer_relevancy_metric, faithfulness_metric, 
         contextual_precision_metric, contextual_recall_metric, contextual_relevancy_metric,
         bias_metric, toxicity_metric, hallucination_metric, 
         RAGAS_answer_relevancy_metric, RAGAS_faithfulness_metric, 
         RAGAS_contextual_precision_metric, RAGAS_contextual_recall_metric, RAGAS_metric]

## 4. Perform evaluation on all rows

In [14]:
test_results = []
for idx, row in trulens_cosinesimilarity_results_df.iterrows():
    test_case = LLMTestCase(
        input=row["question"],
        actual_output=row["RAG_generated_answer"],
        expected_output=row["true_answer"],
        retrieval_context=[row["context"]],
        context=[row["context"]]
    )

    # get the test result
    try:
        results = evaluate(test_cases=[test_case], metrics=metrics)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

    # iterate through the test results
    for test in results.test_results:
        test_data = {
            "test_case": test.name,
            "success": test.success,
            "question": test.input,
            "RAG_generated_answer": test.actual_output,
            "true_answer": test.expected_output,
            "context": test.retrieval_context
        }

        # extract metrics
        for metric in test.metrics_data:
            test_data[f"{metric.name}_score"] = metric.score
            test_data[f"{metric.name}_reason"] = metric.reason
            test_data[f"{metric.name}_success"] = metric.success

        # Append the structured test result
        test_results.append(test_data)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:24, 24.95s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.9, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The score is 0.90 because the irrelevant statements include information about software installation methods, which do not directly address the issue with texinfo and FFTW mentioned in the input., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Great job! There are no contradictions found in the actual output., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The score is 0.00 because irrelevant nodes with 'no' verdicts are not providing information related to the fix of FFTW or the update to version 3.3.4., error: None)
  - ❌ Contextual Recall (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The score is 0.00 because the sentence does not correspond to any part of the retrieval conte

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [15]:
# save the results to a dataframe
deepeval_results_df = pd.DataFrame(test_results)

# merge the results with the trulens and cosine similarity results
rag_results_df = trulens_cosinesimilarity_results_df.merge(deepeval_results_df, 
                                                           on="question", 
                                                           how="inner")
rag_results_df.drop(columns=["test_case", "success", 
                             "true_answer_x", "context_x", "RAG_generated_answer_x",
                             "Answer Relevancy (ragas)_reason", "Faithfulness (ragas)_reason",
                             'Contextual Precision (ragas)_reason', 'Contextual Recall (ragas)_reason'], inplace=True)

rag_results_df.rename(columns={"true_answer_y": "true_answer", 
                               "context_y": "context", 
                               "RAG_generated_answer_y": "RAG_generated_answer"}, inplace=True)
rag_results_df.head()

question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

   trulens_Answer_Relevance  trulens_Groundedness  trulens_Context_Relevance  \
0                  0.000000              0.000000                   0.500000   
1                  0.666667              0.000000                   0.500000   
2                  1.000000              0.555556                   0.666667   
3                  1.000000              0.500000                   0.500000   
4                       NaN                   NaN                        NaN   

   cosine_Answer_Relevance  cosine_Groundedness  cosine_Context_Relevance  \
0                 0.159146             0.314478                  0.602906   
1                 0.545343             0.417874                  0.446407   
2                 0.810223             0.661697                  0.751487   
3                 0.446853             0.696487                  0.606108   
4                 0.297717             0.279427                  0.356063   

   similarity_to_ground_truth  \
0                    0.159146   
1                    0.545343   
2                    0.810223   
3                    0.446853   
4                    0.297717   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer  ...  \
0  Quick comment on the code: \n \n \n \n  petarz...  ...   
1  After several iteration with  @ktl  and  @rowe...  ...   
2  I take this to mean that a DIASource which is ...  ...   
3  Hi James \nmaybe \n dafButler.Butler.get_known...  ...   
4  This has now been fixed and 3.3.4 is the curre...  ...   

  Answer Relevancy (ragas)_success  Faithfulness (ragas)_score  \
0                            False                        0.40   
1                             True                        0.00   
2                             True                        0.00   
3                             True                        0.25   
4                             True                        0.00   

  Faithfulness (ragas)_success  Contextual Precision (ragas)_score  \
0                        False                                 0.0   
1                        False                                 1.0   
2                        False                                 1.0   
3                        False                                 1.0   
4                        False                                 0.0   

   Contextual Precision (ragas)_success Contextual Recall (ragas)_score  \
0                                 False                        0.000000   
1                                  True                        0.333333   
2                                  True                        0.833333   
3                                  True                        0.000000   
4                                 False                        0.000000   

   Contextual Recall (ragas)_success  RAGAS_score RAGAS_reason  RAGAS_success  
0                              False     0.133333         None          False  
1                              False     0.504217         None           True  
2                               True     0.516458         None           True  
3                              False          NaN         None          False  
4                              False     0.178994         None          False  

[5 rows x 46 columns]

## 5.Save the dataframe for reference

In [16]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [17]:
answer_relevancy_metric_cols = ['question', 'context', 'RAG_generated_answer', 'true_answer', 
                                'trulens_Answer_Relevance', 'cosine_Answer_Relevance', 
                                'Answer Relevancy (ragas)_score', 'Answer Relevancy (ragas)_success',
                                'Answer Relevancy_score', 'Answer Relevancy_reason', 'Answer Relevancy_success']
answer_relevancy_rag_results_df = rag_results_df[answer_relevancy_metric_cols]
answer_relevancy_rag_results_df.rename(columns={"trulens_Answer_Relevance": "TruLens",
                                                "cosine_Answer_Relevance": "Cosine_Similarity",
                                                "Answer Relevancy (ragas)_score": "RAGAS_score",
                                                "Answer Relevancy (ragas)_success": "is_RAGAS_threshold_success",
                                                "Answer Relevancy_score": "DeepEval_score",
                                                "Answer Relevancy_reason": "DeepEval_reason",
                                                "Answer Relevancy_success": "is_DeepEval_threshold_success"}, inplace=True)

filename = f"data/results/full/RAG_results_answer_relevancy_{timestamp}.csv"
answer_relevancy_rag_results_df.to_csv(filename, index=False)
answer_relevancy_rag_results_df.head()

/tmp/ipykernel_6229/166881074.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_relevancy_rag_results_df.rename(columns={"trulens_Answer_Relevance": "TruLens",


question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                             context  \
0  [Draft\nLVV-P106: Data Management Acceptance T...   
1  [In most cases, the SWIG files from the curren...   
2  [DPDD | LSE-163 | Latest Revision 2023-07-10\n...   
3  [3   Overview\nThe Butler is implemented as th...   
4  [LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer   TruLens  \
0  Quick comment on the code: \n \n \n \n  petarz...  0.000000   
1  After several iteration with  @ktl  and  @rowe...  0.666667   
2  I take this to mean that a DIASource which is ...  1.000000   
3  Hi James \nmaybe \n dafButler.Butler.get_known...  1.000000   
4  This has now been fixed and 3.3.4 is the curre...       NaN   

   Cosine_Similarity  RAGAS_score  is_RAGAS_threshold_success  DeepEval_score  \
0           0.159146     0.000000                       False        0.833333   
1           0.545343     0.854417                        True        0.666667   
2           0.810223     0.871607                        True        0.875000   
3           0.446853     0.770608                        True        0.875000   
4           0.297717     0.894968                        True        0.900000   

                                     DeepEval_reason  \
0  The score is 0.83 because the statement talks ...   
1  The score is 0.67 because while the actual out...   
2  The score is 0.88 because the statement about ...   
3  The score is 0.88 because the statement is dir...   
4  The score is 0.90 because the irrelevant state...   

   is_DeepEval_threshold_success  
0                           True  
1                          False  
2                           True  
3                           True  
4                           True

In [18]:
groundedness_metric_cols = ['question', 'context', 'RAG_generated_answer', 'true_answer', 
                           'trulens_Groundedness', 'cosine_Groundedness', 
                           'Faithfulness_score', 'Faithfulness_reason','Faithfulness_success',
                           'Faithfulness (ragas)_score', 'Faithfulness (ragas)_success',]
groundedness_metric_rag_results_df = rag_results_df[groundedness_metric_cols]
groundedness_metric_rag_results_df.rename(columns={"trulens_Groundedness": "TruLens",
                                                   "cosine_Groundedness": "Cosine_Similarity",
                                                   "Faithfulness_score": "DeepEval_score",
                                                   "Faithfulness_reason": "DeepEval_reason",
                                                   "Faithfulness_success": "is_DeepEval_threshold_success",
                                                   "Faithfulness (ragas)_score": "RAGAS_score",
                                                   "Faithfulness (ragas)_success": "is_RAGAS_threshold_success"}, inplace=True)

filename = f"data/results/full/RAG_results_groundedness_{timestamp}.csv"
groundedness_metric_rag_results_df.to_csv(filename, index=False)
groundedness_metric_rag_results_df.head()

/tmp/ipykernel_6229/1890442687.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groundedness_metric_rag_results_df.rename(columns={"trulens_Groundedness": "TruLens",


question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                             context  \
0  [Draft\nLVV-P106: Data Management Acceptance T...   
1  [In most cases, the SWIG files from the curren...   
2  [DPDD | LSE-163 | Latest Revision 2023-07-10\n...   
3  [3   Overview\nThe Butler is implemented as th...   
4  [LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer   TruLens  \
0  Quick comment on the code: \n \n \n \n  petarz...  0.000000   
1  After several iteration with  @ktl  and  @rowe...  0.000000   
2  I take this to mean that a DIASource which is ...  0.555556   
3  Hi James \nmaybe \n dafButler.Butler.get_known...  0.500000   
4  This has now been fixed and 3.3.4 is the curre...       NaN   

   Cosine_Similarity  DeepEval_score  \
0           0.314478        0.875000   
1           0.417874        0.666667   
2           0.661697        0.333333   
3           0.696487        0.375000   
4           0.279427        1.000000   

                                     DeepEval_reason  \
0  The score is 0.88 because there is a contradic...   
1  The score is 0.67 because the actual output co...   
2  The score is 0.33 because the actual output co...   
3  The score is 0.38 because the 'actual output' ...   
4  Great job! There are no contradictions found i...   

   is_DeepEval_threshold_success  RAGAS_score  is_RAGAS_threshold_success  
0                           True         0.40                       False  
1                          False         0.00                       False  
2                          False         0.00                       False  
3                          False         0.25                       False  
4                           True         0.00                       False

In [19]:
contextual_metric_cols = ['question', 'context', 'RAG_generated_answer', 'true_answer', 
                           'trulens_Context_Relevance',  'cosine_Context_Relevance',
                           'Contextual Relevancy_score', 'Contextual Relevancy_reason', 'Contextual Relevancy_success',
                           'Contextual Precision (ragas)_score', 'Contextual Precision (ragas)_success',
                           'Contextual Precision_score', 'Contextual Precision_reason', 'Contextual Precision_success',
                           'Contextual Recall_score', 'Contextual Recall_reason', 'Contextual Recall_success', 
                           'Contextual Recall (ragas)_score', 'Contextual Recall (ragas)_success']
contextual_metric_rag_results_df = rag_results_df[contextual_metric_cols]
contextual_metric_rag_results_df.rename(columns={"trulens_Context_Relevance":"TruLens_Context_Relevance",
                                                "cosine_Context_Relevance":"Cosine_Similarity_Context_Relevance"}, inplace=True)

filename = f"data/results/full/RAG_results_contextual_metrics_{timestamp}.csv"
contextual_metric_rag_results_df.to_csv(filename, index=False)
contextual_metric_rag_results_df.head()

/tmp/ipykernel_6229/44955328.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contextual_metric_rag_results_df.rename(columns={"trulens_Context_Relevance":"TruLens_Context_Relevance",


question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                             context  \
0  [Draft\nLVV-P106: Data Management Acceptance T...   
1  [In most cases, the SWIG files from the curren...   
2  [DPDD | LSE-163 | Latest Revision 2023-07-10\n...   
3  [3   Overview\nThe Butler is implemented as th...   
4  [LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer  \
0  Quick comment on the code: \n \n \n \n  petarz...   
1  After several iteration with  @ktl  and  @rowe...   
2  I take this to mean that a DIASource which is ...   
3  Hi James \nmaybe \n dafButler.Butler.get_known...   
4  This has now been fixed and 3.3.4 is the curre...   

   TruLens_Context_Relevance  Cosine_Similarity_Context_Relevance  \
0                   0.500000                             0.602906   
1                   0.500000                             0.446407   
2                   0.666667                             0.751487   
3                   0.500000                             0.606108   
4                        NaN                             0.356063   

   Contextual Relevancy_score  \
0                    0.000000   
1                    0.333333   
2                    0.750000   
3                    0.000000   
4                    0.333333   

                         Contextual Relevancy_reason  \
0  The score is 0.00 because the input provided d...   
1  The score is 0.33 because the high-level infor...   
2  The score is 0.75 because the input addresses ...   
3  The score is 0.00 because the input is focused...   
4  The score is 0.33 because while there is relev...   

   Contextual Relevancy_success  Contextual Precision (ragas)_score  \
0                         False                                 0.0   
1                         False                                 1.0   
2                          True                                 1.0   
3                         False                                 1.0   
4                         False                                 0.0   

   Contextual Precision (ragas)_success  Contextual Precision_score  \
0                                 False                         1.0   
1                                  True                         0.0   
2                                  True                         1.0   
3                                  True                         0.0   
4                                 False                         0.0   

                         Contextual Precision_reason  \
0  The score is 1.00 because the retrieval contex...   
1  The score is 0.00 because the only retrieval c...   
2  The score is 1.00 because all relevant retriev...   
3  The score is 0.00 because the retrieval contex...   
4  The score is 0.00 because irrelevant nodes wit...   

   Contextual Precision_success  Contextual Recall_score  \
0                          True                 1.000000   
1                         False                 0.200000   
2                          True                 0.833333   
3                         False                 0.428571   
4                         False                 0.000000   

                            Contextual Recall_reason  \
0  The score is 1.00 because all sentences in the...   
1  The score is 0.20 because the supportive reaso...   
2  The score is 0.83 beca

In [20]:
cosine_similarity_ground_truth_cols = ['question', 'context', 'RAG_generated_answer', 'true_answer', 'similarity_to_ground_truth']
cosine_similarity_ground_truth_df = rag_results_df[cosine_similarity_ground_truth_cols]
cosine_similarity_ground_truth_df.rename(columns={"'similarity_to_ground_truth'":"cosine_similarity_true_answer_ground_truth'"}, inplace=True)

filename = f"data/results/full/RAG_results_cosine_similarity_ground_truth_{timestamp}.csv"
cosine_similarity_ground_truth_df.to_csv(filename, index=False)
cosine_similarity_ground_truth_df.head()

/tmp/ipykernel_6229/1570021341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosine_similarity_ground_truth_df.rename(columns={"'similarity_to_ground_truth'":"cosine_similarity_true_answer_ground_truth'"}, inplace=True)


question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                             context  \
0  [Draft\nLVV-P106: Data Management Acceptance T...   
1  [In most cases, the SWIG files from the curren...   
2  [DPDD | LSE-163 | Latest Revision 2023-07-10\n...   
3  [3   Overview\nThe Butler is implemented as th...   
4  [LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer  \
0  Quick comment on the code: \n \n \n \n  petarz...   
1  After several iteration with  @ktl  and  @rowe...   
2  I take this to mean that a DIASource which is ...   
3  Hi James \nmaybe \n dafButler.Butler.get_known...   
4  This has now been fixed and 3.3.4 is the curre...   

   similarity_to_ground_truth  
0                    0.159146  
1                    0.545343  
2                    0.810223  
3                    0.446853  
4                    0.297717

In [21]:
other_deepeval_metric_cols = ['question', 'context', 'RAG_generated_answer', 'true_answer', 
                            'Bias_score', 'Bias_reason', 'Bias_success', 
                            'Toxicity_score', 'Toxicity_reason', 'Toxicity_success', 
                            'Hallucination_score', 'Hallucination_reason', 'Hallucination_success']
other_deepeval_metric_rag_results_df = rag_results_df[other_deepeval_metric_cols]

filename = f"data/results/full/RAG_results_other_deepeval_metrics_{timestamp}.csv"
other_deepeval_metric_rag_results_df.to_csv(filename, index=False)
other_deepeval_metric_rag_results_df.head()

question  \
0  Hi, \nI’m following this tutorial:  The LSST S...   
1  I have the following C++ class : \n class CcdI...   
2  Question on how forced photometry will be run ...   
3  Hi there, \n Is there some way I find out what...   
4  I’m having trouble building FFTW with texinfo ...   

                                             context  \
0  [Draft\nLVV-P106: Data Management Acceptance T...   
1  [In most cases, the SWIG files from the curren...   
2  [DPDD | LSE-163 | Latest Revision 2023-07-10\n...   
3  [3   Overview\nThe Butler is implemented as th...   
4  [LARGE SYNOPTIC SURVEY TELESCOPE\nNotes on use...   

                                RAG_generated_answer  \
0  ```go\n\nAnswer:\n\nThe behavior you observe i...   
1  \nAnswer:\nTo make the CcdImageList iterable i...   
2  \nAnswer: Forced photometry measurements with ...   
3  \nAnswer:\nThe 'butler' object created in line...   
4  \nAnswer: The known issue with Texinfo and FFT...   

                                         true_answer  Bias_score  \
0  Quick comment on the code: \n \n \n \n  petarz...         0.0   
1  After several iteration with  @ktl  and  @rowe...         0.0   
2  I take this to mean that a DIASource which is ...         0.0   
3  Hi James \nmaybe \n dafButler.Butler.get_known...         0.0   
4  This has now been fixed and 3.3.4 is the curre...         0.0   

                                         Bias_reason  Bias_success  \
0  The score is 0.00 because there are no reasons...          True   
1  The score is 0.00 because there are no reasons...          True   
2  The score is 0.00 because there are no reasons...          True   
3  The score is 0.00 because there are no reasons...          True   
4  The score is 0.00 because there are no reasons...          True   

   Toxicity_score                                    Toxicity_reason  \
0             0.0  The score is 0.00 because there are no reasons...   
1             0.0  The score is 0.00 because there are no reasons...   
2             0.0  The score is 0.00 because there are no reasons...   
3             0.0  The score is 0.00 because there are no reasons...   
4             0.0  The score is 0.00 because there are no instanc...   

   Toxicity_success  Hallucination_score  \
0              True                  1.0   
1              True                  1.0   
2              True                  0.5   
3              True                  1.0   
4              True                  1.0   

                                Hallucination_reason  Hallucination_success  
0  The score is 1.00 because there are no factual...                  False  
1  The score is 1.00 because there are contradict...                  False  
2  The score is 0.50 because the actual output al...                   True  
3  The score is 1.00 because the actual output do...                  False  
4  The score is 1.00 because the actual output gr...                  False